In [ ]:
import geopandas as gpd
from geopandas.geoseries import *
import numpy as np

# source: Amt für Statistik Berlin-Brandenburg: Postleitzahlen
# https://fbinter.stadt-berlin.de/fb/index.jsp?loginkey=zoomStart&mapId=plz@senstadt&bbox=367109,5805216,418853,5832317
plz = gpd.read_file('source/plz.geojson')
print(len(plz)) # 190 PLZs

plr = gpd.read_file('source/LOR_Planungsraeume_2019.geojson')
plr_data = pd.read_csv('source/LOR_Planungsraeume_2019.csv', sep=';')

plr_data.rename(columns={'Planungsraum': 'PLANUNGSRAUM', 'Bezirksregion': 'BEZIRKSREGION'}, inplace=True)

plr = plr.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])

df = pd.DataFrame(columns = ['PLR', 'PLR_NAME', 'PLZ'])
df['PLZ'] = np.empty((len(plr), 0)).tolist()
df.head()
for indexPLR, rowPLR in plr.iterrows():
    df.loc[indexPLR, 'PLR'] = rowPLR.Schlüssel
    df.loc[indexPLR, 'PLR_NAME'] = rowPLR.PLANUNGSRAUM
    for indexPLZ, rowPLZ in plz.iterrows():
        if rowPLZ.geometry.intersects(rowPLR.geometry):
            df.loc[indexPLR, 'PLZ'].append(str(rowPLZ.spatial_name))

df.to_csv('output/PLR2PLZ.csv')



df2 = pd.DataFrame(columns = ['PLZ', 'PLR'])
df2['PLR'] = np.empty((len(plz), 0)).tolist()

for indexPLZ, rowPLZ in plz.iterrows():
    df2.loc[indexPLZ, 'PLZ'] = rowPLZ.spatial_name
    for indexPLR, rowPLR in plr.iterrows():
        if rowPLR.geometry.intersects(rowPLZ.geometry):
            df2.loc[indexPLZ, 'PLR'].append(rowPLR.Schlüssel)

df2.to_csv('output/PLZ2PLR.csv')

190
